In [1]:
using ComponentArrays, Lux, DiffEqFlux, OrdinaryDiffEq, Optimization, OptimizationOptimJL,
      OptimizationOptimisers, Random, Plots
using ArrheniusModel

In [2]:
function arrhenius_rate(pe::PhaseEnergies, T=300)
    kb = 8.617e-5 #eV/K
    A = 1.0 # Arrhenius prefactor
    pe.K = A * exp.(-pe.Ea_plus_ΔG ./ (kb * T))
    # Adjust the diagonal elements
    for i in 1:size(pe.K, 1)
        pe.K[i, i] =  -1 * sum(pe.K[i, [1:i-1; i+1:end]])
    end
end

arrhenius_rate (generic function with 2 methods)

In [3]:
rng = Xoshiro(0)
G = [-5.92, -5.942, -5.97]
Ea = [0.00 1.00 0.01; 1.00 0.00 1.00; 0.01 1.00 0.00]
pe = PhaseEnergies(G, Ea)
T = 300.0
t= 10
dt = 0.05
num_steps = floor(Int, t/dt)
num_layers = floor(Int, t/0.5)+1
flow_rate = 0.5
decay_coefficient = 0.00001 * flow_rate
fcoeff = flow_coefficient("exponential", num_layers, decay_coefficient)

function deposition_rates!(dc, c, p, t)
    # Unpack parameters
    fcoeff, pe, j0, j, dt, num_steps, num_layers = p
    # Calculate deposition rates
    j = floor(Int, t / 0.5) + 1
    f = reverse(fcoeff[j: num_layers+j-1])
    dc .= c .* f * pe.K
    if j != j0
        c[j+1, 1] = 1.0
        j = j0
    end
end

n = n_phases(pe)
c0 = zeros(num_layers, n)
c0[1, 1] = 1.0
arrhenius_rate(pe, T)
j = 0
j0 = 0
p = (fcoeff, pe, j0, j, dt, num_steps, num_layers)
tspan = (0.0, (num_steps-1) * dt)
prob = ODEProblem(deposition_rates!, c0, tspan, p)
ode_data = Array(solve(prob, Euler(), saveat = 0.5, dt = dt))


21×3×21 Array{Float64, 3}:
[:, :, 1] =
 1.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 ⋮         
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

[:, :, 2] =
 0.714042      7.20251e-18   0.285958
 1.0          -9.76963e-49  -4.1859e-32
 3.64848e-30   0.0           0.0
 0.0           0.0           0.0
 0.0           0.0           0.0
 0.0           0.0           0.0
 0.0           0.0           0.0
 0.0           0.0           0.0
 0.0           0.0           0.0
 0.0           0.0           0.0
 ⋮                          
 0.0           0.0           0.0
 0.0           0.0           0.0
 0.0           0.0           0.0
 0.0           0.0           0.0
 0.0           0.0           0.0
 0.0           0.0           0.0
 0.0           0.0           0.0
 0.0           0.0           0.0
 0.0           0

In [28]:
display(ode_data[1])

1.0

In [8]:
inputs = [T, flow_rate]
input_size = length(inputs)  # Replace with the actual size of `inputs` if it's not a 1D vector
fcoeff_size = length(fcoeff)
output_size = n ^ 2 + fcoeff_size + 1
dudt2 = Chain(
    BatchNorm(input_size),
    Dense(2, 21, relu),  # First layer, adjust `input_size` accordingly
    Dense(21, 63, relu),         # Intermediate layer
    Dense(63, output_size)       # Output layer, adjust `output_size` accordingly
)

p, st = Lux.setup(rng, dudt2)
prob_neuralode = NeuralODE(dudt2, tspan, Euler(); saveat = 0.5, dt=dt)

NeuralODE(
    model = Chain(
        layer_1 = BatchNorm(2, affine=true, track_stats=true),  # 4 parameters, plus 5
        layer_2 = Dense(2 => 21, relu),  # 63 parameters
        layer_3 = Dense(21 => 63, relu),  # 1_386 parameters
        layer_4 = Dense(63 => 51),      # 3_264 parameters
    ),
)         # Total: 4_717 parameters,
          #        plus 5 states.

In [9]:
function predict_neuralode(p)
    Array(prob_neuralode(c0, p, st)[1])
end

function loss_neuralode(p)
    pred = predict_neuralode(p)
    loss = sum(abs2, ode_data .- pred)
    return loss, pred
end

loss_neuralode (generic function with 1 method)

In [10]:
# Callback function to observe training
callback = function (p, l, pred; doplot = false)
    println(l)
    # plot current prediction against data
    if doplot
        plt = scatter(tsteps, ode_data[1, :]; label = "data")
        scatter!(plt, tsteps, pred[1, :]; label = "prediction")
        display(plot(plt))
    end
    return false
end

pinit = ComponentArray(p)
callback(pinit, loss_neuralode(pinit)...)

ArgumentError: ArgumentError: Invalid Dimensions!

In [7]:
# Train using the Adam optimizer
adtype = Optimization.AutoZygote()

optf = Optimization.OptimizationFunction((x, p) -> loss_neuralode(x), adtype)
optprob = Optimization.OptimizationProblem(optf, pinit)

result_neuralode = Optimization.solve(
    optprob, OptimizationOptimisers.Adam(0.05); callback = callback, maxiters = 300)

DimensionMismatch: DimensionMismatch: matrix A has dimensions (64,3), matrix B has dimensions (21,3)

In [8]:
# Retrain using the LBFGS optimizer
optprob2 = remake(optprob; u0 = result_neuralode.u)

result_neuralode2 = Optimization.solve(optprob2, Optim.BFGS(; initial_stepnorm = 0.01);
    callback = callback, allow_f_increases = false)

UndefVarError: UndefVarError: `result_neuralode` not defined

In [9]:
callback(result_neuralode2.u, loss_neuralode(result_neuralode2.u)...; doplot = true)
scatter!(plt, tsteps, loss_neuralode(result_neuralode2.u)[2][1, :]; label = "prediction") # hide

UndefVarError: UndefVarError: `result_neuralode2` not defined